In [1]:
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np

In [3]:
matches = pd.read_csv("C:/Users/jenil/OneDrive/Desktop/Jupyter/Run Predication/matches.csv")
delivery = pd.read_csv("C:/Users/jenil/OneDrive/Desktop/Jupyter/Run Predication/deliveries.csv")

In [4]:
matches.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan


In [5]:
matches['target_runs'] = matches['target_runs'] - 1
matches.rename(columns = {'target_runs' : 'total_runs'}, inplace = True)
matches.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,total_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,222.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,240.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,129.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,165.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,110.0,20.0,N,NaN,BF Bowden,K Hariharan


In [6]:
teams = {
    'Chennai Super Kings',
    'Mumbai Indians',
    'Delhi Capitals',
    'Kolkata Knight Riders',
    'Punjab Kings',
    'Royal Challengers Bengaluru',
    'Sunrisers Hyderabad',
    'Rajasthan Royals',
    'Lucknow Super Giants',
    'Gujarat Titans'
}

In [7]:
matches['team1'] = matches['team1'].str.replace('Gujarat Lions', 'Gujarat Titans')
matches['team2'] = matches['team2'].str.replace('Gujarat Lions', 'Gujarat Titans')

matches['team1'] = matches['team1'].str.replace('Delhi Daredevils', 'Delhi Capitals')
matches['team2'] = matches['team2'].str.replace('Delhi Daredevils', 'Delhi Capitals')

matches['team1'] = matches['team1'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')
matches['team2'] = matches['team2'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')

matches['team1'] = matches['team1'].str.replace('Kings XI Punjab', 'Punjab Kings')
matches['team2'] = matches['team2'].str.replace('Kings XI Punjab', 'Punjab Kings')

In [8]:
matches = matches[matches['team1'].isin(teams)]
matches = matches[matches['team2'].isin(teams)]

In [9]:
matches.shape

(777, 20)

In [10]:
matches = matches[['id', 'city', 'total_runs']]
matches.head()

,id,city,total_runs
1,335983,Chandigarh,240.0
2,335984,Delhi,129.0
4,335986,Kolkata,110.0
5,335987,Jaipur,166.0
6,335988,Hyderabad,142.0


In [11]:
delivery = delivery[delivery['inning'] != 2]

In [12]:
delivery['batting_team'] = delivery['batting_team'].str.replace('Gujarat Lions', 'Gujarat Titans')
delivery['bowling_team'] = delivery['bowling_team'].str.replace('Gujarat Lions', 'Gujarat Titans')

delivery['batting_team'] = delivery['batting_team'].str.replace('Delhi Daredevils', 'Delhi Capitals')
delivery['bowling_team'] = delivery['bowling_team'].str.replace('Delhi Daredevils', 'Delhi Capitals')

delivery['batting_team'] = delivery['batting_team'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')
delivery['bowling_team'] = delivery['bowling_team'].str.replace('Deccan Chargers', 'Sunrisers Hyderabad')

delivery['batting_team'] = delivery['batting_team'].str.replace('Kings XI Punjab', 'Punjab Kings')
delivery['bowling_team'] = delivery['bowling_team'].str.replace('Kings XI Punjab', 'Punjab Kings')

In [13]:
delivery = delivery[delivery['batting_team'].isin(teams)]
delivery = delivery[delivery['bowling_team'].isin(teams)]

In [14]:
delivery = matches.merge(delivery, left_on = 'id', right_on = 'match_id')

In [15]:
delivery.head()

,id,city,total_runs_x,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs_y,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335983,Chandigarh,240.0,335983,1,Chennai Super Kings,Punjab Kings,0,1,PA Patel,B Lee,ML Hayden,0,0,0,NaN,0,NaN,NaN,NaN
1,335983,Chandigarh,240.0,335983,1,Chennai Super Kings,Punjab Kings,0,2,PA Patel,B Lee,ML Hayden,0,0,0,NaN,0,NaN,NaN,NaN
2,335983,Chandigarh,240.0,335983,1,Chennai Super Kings,Punjab Kings,0,3,PA Patel,B Lee,ML Hayden,1,0,1,NaN,0,NaN,NaN,NaN
3,335983,Chandigarh,240.0,335983,1,Chennai Super Kings,Punjab Kings,0,4,ML Hayden,B Lee,PA Patel,0,0,0,NaN,0,NaN,NaN,NaN
4,335983,Chandigarh,240.0,335983,1,Chennai Super Kings,Punjab Kings,0,5,ML Hayden,B Lee,PA Patel,4,0,4,NaN,0,NaN,NaN,NaN


In [16]:
delivery['ball_left'] = 120 - (delivery['over']*6 + delivery['ball'])

delivery.rename(columns = {'total_runs' : 'ct_runs'}, inplace = True)

delivery['is_wicket'] = delivery['is_wicket'].astype(int)
wicket = delivery.groupby('match_id')['is_wicket'].cumsum()

delivery['wicket_left'] = 10 - wicket

delivery['current_run'] = delivery.groupby('id')['total_runs_y'].cumsum()

delivery['crr'] = delivery['current_run']*6 / ((120 - delivery['ball_left']))

delivery['last_5_overs'] = delivery.groupby('id')['total_runs_y'].transform(lambda x: x.rolling(30).sum())

In [17]:
delivery.head()

,id,city,total_runs_x,match_id,inning,batting_team,bowling_team,over,ball,batter,...,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,ball_left,wicket_left,current_run,crr,last_5_overs
0,335983,Chandigarh,240.0,335983,1,Chennai Super Kings,Punjab Kings,0,1,PA Patel,...,NaN,0,NaN,NaN,NaN,119,10,0,0.0,NaN
1,335983,Chandigarh,240.0,335983,1,Chennai Super Kings,Punjab Kings,0,2,PA Patel,...,NaN,0,NaN,NaN,NaN,118,10,0,0.0,NaN
2,335983,Chandigarh,240.0,335983,1,Chennai Super Kings,Punjab Kings,0,3,PA Patel,...,NaN,0,NaN,NaN,NaN,117,10,1,2.0,NaN
3,335983,Chandigarh,240.0,335983,1,Chennai Super Kings,Punjab Kings,0,4,ML Hayden,...,NaN,0,NaN,NaN,NaN,116,10,1,1.5,NaN
4,335983,Chandigarh,240.0,335983,1,Chennai Super Kings,Punjab Kings,0,5,ML Hayden,...,NaN,0,NaN,NaN,NaN,115,10,5,6.0,NaN


In [18]:
df = delivery[['batting_team', 'bowling_team', 'current_run', 'city', 'ball_left', 'last_5_overs', 'crr', 'total_runs_x', 'wicket_left']]

In [19]:
df.head()

,batting_team,bowling_team,current_run,city,ball_left,last_5_overs,crr,total_runs_x,wicket_left
0,Chennai Super Kings,Punjab Kings,0,Chandigarh,119,NaN,0.0,240.0,10
1,Chennai Super Kings,Punjab Kings,0,Chandigarh,118,NaN,0.0,240.0,10
2,Chennai Super Kings,Punjab Kings,1,Chandigarh,117,NaN,2.0,240.0,10
3,Chennai Super Kings,Punjab Kings,1,Chandigarh,116,NaN,1.5,240.0,10
4,Chennai Super Kings,Punjab Kings,5,Chandigarh,115,NaN,6.0,240.0,10


In [20]:
df['city'].unique()

array(['Chandigarh', 'Delhi', 'Kolkata', 'Jaipur', 'Hyderabad', 'Chennai',
       'Mumbai', 'Cape Town', 'Durban', 'Port Elizabeth', 'Centurion',
       'East London', 'Johannesburg', 'Kimberley', 'Bloemfontein',
       'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala', 'Visakhapatnam',
       'Pune', 'Bangalore', 'Raipur', 'Abu Dhabi', nan, 'Ranchi',
       'Rajkot', 'Kanpur', 'Indore', 'Dubai', 'Sharjah', 'Navi Mumbai',
       'Lucknow', 'Guwahati', 'Mohali', 'Bengaluru'], dtype=object)

In [21]:
df.isnull().sum()

batting_team        0
bowling_team        0
current_run         0
city             4619
ball_left           0
last_5_overs    22533
crr                 0
total_runs_x      118
wicket_left         0
dtype: int64

In [22]:
df.dropna(inplace = True)

C:\Users\jenil\AppData\Local\Temp\ipykernel_9596\1791587065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace = True)


In [23]:
df.shape

(70221, 9)

In [24]:
x = df.drop(columns = ['total_runs_x'])
y = df[['total_runs_x']]

In [25]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, TargetEncoder

categorical_features = ['batting_team', 'bowling_team', 'city']

te = ColumnTransformer([
    ('te', TargetEncoder(target_type= 'continuous'),categorical_features)
      ],  remainder = 'passthrough')

In [41]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

pipe = Pipeline(steps = [
    ('step1', te),
    ('step2', XGBRegressor(n_estimators=200,
        max_depth=5,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42))
])

In [42]:
pipe.fit(x_train, y_train.values.ravel())

D:\AnA\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('te',
                                                  TargetEncoder(target_type='continuous'),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.8, device=None,
                              early_stopping_rounds=None,
                              enable_ca...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=5, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=200, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [43]:
y_pred = pipe.predict(x_test)

In [44]:
mae = mean_absolute_error(y_test.values.ravel(), y_pred)
mse = mean_squared_error(y_test.values.ravel(), y_pred)
r2  = r2_score(y_test.values.ravel(), y_pred)

In [45]:
print("MAE:", mae)
print("MSE:", mse)
print("R²:", r2)

MAE: 10.690066591875967
MSE: 245.90334961596886
R²: 0.7725663665959868


In [46]:
import pickle

In [48]:
filename = "C:/Users/jenil/OneDrive/Desktop/Jupyter/Run Predication/run.pkl"
pickle.dump(pipe, open(filename, "wb"))

In [49]:
df['city'].unique()

array(['Chandigarh', 'Delhi', 'Kolkata', 'Jaipur', 'Hyderabad', 'Chennai',
       'Mumbai', 'Cape Town', 'Durban', 'Port Elizabeth', 'Centurion',
       'East London', 'Johannesburg', 'Kimberley', 'Bloemfontein',
       'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala', 'Visakhapatnam',
       'Pune', 'Bangalore', 'Raipur', 'Abu Dhabi', 'Ranchi', 'Rajkot',
       'Kanpur', 'Indore', 'Dubai', 'Sharjah', 'Navi Mumbai', 'Lucknow',
       'Guwahati', 'Mohali', 'Bengaluru'], dtype=object)